In [114]:
%load_ext autoreload
#To update the imported files

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
%autoreload
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error
from utilities import *
import seaborn as sns
%matplotlib inline


pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [116]:
#Loading the y-paramter from file, might be changed to y_a etc.
train_a = pd.read_parquet('../Data_and_task/A/train_targets.parquet')
train_b = pd.read_parquet('../Data_and_task/B/train_targets.parquet')
train_c = pd.read_parquet('../Data_and_task/C/train_targets.parquet')



In [117]:
#Loading estimated/forecasted training_weather from file
X_train_estimated_a = pd.read_parquet('../Data_and_task/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../Data_and_task/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../Data_and_task/C/X_train_estimated.parquet')

In [118]:
#Loading observed weather from file
X_train_observed_a = pd.read_parquet('../Data_and_task/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../Data_and_task/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../Data_and_task/C/X_train_observed.parquet')

In [119]:
#Loading estimated/forecasted test_weather from file
X_test_estimated_a = pd.read_parquet('../Data_and_task/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../Data_and_task/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../Data_and_task/C/X_test_estimated.parquet')

In [70]:
#X_train_estimated_a.head()
#X_test_estimated_b.head()
#X_test_estimated_c.head()

In [120]:
#Cleaning data such that only wanted features are included
selected_features = [
    "date_forecast", "clear_sky_rad:W", "diffuse_rad:W", "direct_rad:W",
    "absolute_humidity_2m:gm3", "effective_cloud_cover:p",
    "is_day:idx", "is_in_shadow:idx", "fresh_snow_12h:cm", "fresh_snow_1h:cm",
    "fresh_snow_24h:cm", "fresh_snow_3h:cm", "fresh_snow_6h:cm",
    "snow_depth:cm", "snow_melt_10min:mm", "sun_azimuth:d", 
    "sun_elevation:d", "wind_speed_10m:ms"
]

#Defining y_fetures, will always be the same
y_features = ["time", "pv_measurement"]

X_train_observed_a_clean = X_train_observed_a[selected_features].copy()
X_train_estimated_a_clean = X_train_estimated_a[selected_features].copy()

In [121]:
#calculating average of x value 
#NB this calculation takes a long time, and one should feture engineer all dataframes before running to 
#shorten the runtime
X_train_observed_a_average = X_train_observed_a_clean.groupby(X_train_observed_a_clean.index // 4).apply(custom_mean2).reset_index(drop=True)
#X_train_observed_b_average = X_train_observed_b.groupby(X_train_observed_b.index // 4).apply(custom_mean2).reset_index(drop=True)
#X_train_observed_c_average = X_train_observed_c.groupby(X_train_observed_c.index // 4).apply(custom_mean2).reset_index(drop=True)
#check when finished
print("calculation complete")

calculation complete


In [122]:
X_train_estimated_a_average = X_train_estimated_a_clean.groupby(X_train_estimated_a_clean.index // 4).apply(custom_mean2).reset_index(drop=True)
X_train_estimated_b_average = X_test_estimated_b.groupby(X_test_estimated_b.index // 4).apply(custom_mean2).reset_index(drop=True)
X_train_estimated_c_average = X_test_estimated_c.groupby(X_test_estimated_c.index // 4).apply(custom_mean2).reset_index(drop=True)
#check when finished
print("calculation complete")

calculation complete


In [83]:
#export data to exel file
file_name2 = "X_train_observed_a_average.xlsx" 
# saving the excel
X_train_observed_a_average.to_excel(file_name2)

In [129]:
X_test_estimated_a_clean = X_train_estimated_a_average.drop(columns = ["date_forecast"]).copy()
X_test_estimated_b_clean = X_train_estimated_b_average.drop(columns = ["date_forecast"]).copy()
X_test_estimated_c_clean = X_train_estimated_c_average.drop(columns = ["date_forecast"]).copy()

In [125]:
#Training a Linear regression model on X_observed_a and testing it on X_estimated_a and evaluating it on MAE, PURELY for testing!
#See below for how its done when submitting for A
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_average, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_average, train_a, "date_forecast", y_features)

#print(X_train_observed_a_resized)
#print(y_train_observed_a)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_a_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_a_resized)

#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, y_train_observed_a["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(y_train_estimated_a["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 370.14963323421324


In [126]:
#for b
X_train_observed_b_resized, y_train_observed_b = resize_trainingdata(X_train_observed_b_clean, train_b, "date_forecast", y_features)
X_train_estimated_b_resized, y_train_estimated_b = resize_trainingdata(X_train_estimated_b_clean, train_b, "date_forecast", y_features)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_b_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_b_resized)

y_train_observed_b = remove_NaN_from_Y(y_train_observed_b,"pv_measurement")
#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, y_train_observed_b["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(y_train_estimated_b["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)

NameError: name 'X_train_observed_b_clean' is not defined

In [105]:
X_train_observed_a_average.shape


(29668, 18)

In [132]:
#Training model, running infernece on testing data and saving it in a csv file ready for submission
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_average, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_average, train_a, "date_forecast", y_features)

#Merging observed and estimated data to one big df
X_train_a = pd.concat([X_train_observed_a_resized, X_train_estimated_a_resized], ignore_index=True)
y_train_a = pd.concat([y_train_observed_a, y_train_estimated_a], ignore_index=True)
X_test = pd.concat([X_test_estimated_a_clean, X_test_estimated_b_clean, X_test_estimated_c_clean], ignore_index=True)
print(X_test)
#display(X_test)
#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_a_scaled = scaler.fit_transform(X_train_a)
X_test_a_scaled=scaler.fit_transform(X_test)

#Training the models
#Model A:
reg_a = LinearRegression()
reg_a.fit(X_train_a, y_train_a["pv_measurement"])

# Make predictions for A,B,C on the model trained on only A. Problems with the B and C datasets which need to be explored
y_pred = reg_a.predict(X_test_a_scaled)

#Saving y_pred in a proper csv file
filename = "CSV/second_submission.csv"
submission(filename, y_pred)

      clear_sky_rad:W  diffuse_rad:W  direct_rad:W  absolute_humidity_2m:gm3  \
0            0.000000       0.000000      0.000000                  8.349999   
1            0.000000       0.000000      0.000000                  8.100000   
2            0.000000       0.000000      0.000000                  8.150000   
3            0.000000       0.000000      0.000000                  8.200000   
4            0.000000       0.000000      0.000000                  8.375000   
...               ...            ...           ...                       ...   
5829        84.775002      32.200001     13.450001                  8.400000   
5830        24.550001      13.875000      2.400000                  8.600000   
5831         1.225000       1.250000      0.000000                  8.875000   
5832         0.000000       0.000000      0.000000                  9.000000   
5833         0.000000       0.000000      0.000000                  8.950000   

      effective_cloud_cover:p  is_day:i

C:\Users\Gustav\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [110]:
y_train_estimated_a.shape

(4394, 2)